In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import random

from sklearn.metrics import classification_report
import os
from sklearn.model_selection import train_test_split

import tensorflow as tf

**Exploring the Dataset**

In [ ]:
data=pd.read_csv("ner_dataset.csv",encoding="latin1")
data.head(10)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
5,NaN,through,IN,O
6,NaN,London,NNP,B-geo
7,NaN,to,TO,O
8,NaN,protest,VB,O
9,NaN,the,DT,O


In [ ]:
data['Sentence #']=data['Sentence #'].ffill(axis = 0)
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O


In [ ]:
agg_func = lambda s: [(w,p, t) for w,p, t in zip(s["Word"].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                        s["Tag"].values.tolist())]

In [ ]:
agg_data=data.groupby(['Sentence #']).apply(agg_func).reset_index().rename(columns={0:'Sentence_POS_Tag_Pair'})
agg_data.head()

,Sentence #,Sentence_POS_Tag_Pair
0,Sentence: 1,"[(Thousands, NNS, O), (of, IN, O), (demonstrat..."
1,Sentence: 10,"[(Iranian, JJ, B-gpe), (officials, NNS, O), (s..."
2,Sentence: 100,"[(Helicopter, NN, O), (gunships, NNS, O), (Sat..."
3,Sentence: 1000,"[(They, PRP, O), (left, VBD, O), (after, IN, O..."
4,Sentence: 1001,"[(The, DT, O), (global, JJ, O), (financial, JJ..."


In [ ]:
agg_data['Sentence']=agg_data['Sentence_POS_Tag_Pair'].apply(lambda sentence:" ".join([s[0] for s in sentence]))
agg_data['POS']=agg_data['Sentence_POS_Tag_Pair'].apply(lambda sentence:" ".join([s[1] for s in sentence]))
agg_data['Tag']=agg_data['Sentence_POS_Tag_Pair'].apply(lambda sentence:" ".join([s[2] for s in sentence]))

In [ ]:
agg_data.head()

,Sentence #,Sentence_POS_Tag_Pair,Sentence,POS,Tag
0,Sentence: 1,"[(Thousands, NNS, O), (of, IN, O), (demonstrat...",Thousands of demonstrators have marched throug...,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP C...,O O O O O O B-geo O O O O O B-geo O O O O O B-...
1,Sentence: 10,"[(Iranian, JJ, B-gpe), (officials, NNS, O), (s...",Iranian officials say they expect to get acces...,JJ NNS VBP PRP VBP TO VB NN TO JJ JJ NNS IN DT...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...
2,Sentence: 100,"[(Helicopter, NN, O), (gunships, NNS, O), (Sat...",Helicopter gunships Saturday pounded militant ...,"NN NNS NNP VBD JJ NNS IN DT NNP JJ NN , WRB JJ...",O O B-tim O O O O O B-geo O O O O O B-org O O ...
3,Sentence: 1000,"[(They, PRP, O), (left, VBD, O), (after, IN, O...",They left after a tense hour-long standoff wit...,PRP VBD IN DT NN JJ NN IN NN NNS .,O O O O O O O O O O O
4,Sentence: 1001,"[(The, DT, O), (global, JJ, O), (financial, JJ...",The global financial crisis has left Iceland '...,DT JJ JJ NN VBZ VBN NNP POS NN IN NNS .,O O O O O O B-org O O O O O


In [ ]:
agg_data.shape

(9968, 5)

In [ ]:
agg_data['tokenised_sentences']=agg_data['Sentence'].apply(lambda x:x.split())
agg_data['tag_list']=agg_data['Tag'].apply(lambda x:x.split())
agg_data.head()

,Sentence #,Sentence_POS_Tag_Pair,Sentence,POS,Tag,tokenised_sentences,tag_list
0,Sentence: 1,"[(Thousands, NNS, O), (of, IN, O), (demonstrat...",Thousands of demonstrators have marched throug...,NNS IN NNS VBP VBN IN NNP TO VB DT NN IN NNP C...,O O O O O O B-geo O O O O O B-geo O O O O O B-...,"[Thousands, of, demonstrators, have, marched, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo..."
1,Sentence: 10,"[(Iranian, JJ, B-gpe), (officials, NNS, O), (s...",Iranian officials say they expect to get acces...,JJ NNS VBP PRP VBP TO VB NN TO JJ JJ NNS IN DT...,B-gpe O O O O O O O O O O O O O O B-tim O O O ...,"[Iranian, officials, say, they, expect, to, ge...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
2,Sentence: 100,"[(Helicopter, NN, O), (gunships, NNS, O), (Sat...",Helicopter gunships Saturday pounded militant ...,"NN NNS NNP VBD JJ NNS IN DT NNP JJ NN , WRB JJ...",O O B-tim O O O O O B-geo O O O O O B-org O O ...,"[Helicopter, gunships, Saturday, pounded, mili...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O..."
3,Sentence: 1000,"[(They, PRP, O), (left, VBD, O), (after, IN, O...",They left after a tense hour-long standoff wit...,PRP VBD IN DT NN JJ NN IN NN NNS .,O O O O O O O O O O O,"[They, left, after, a, tense, hour-long, stand...","[O, O, O, O, O, O, O, O, O, O, O]"
4,Sentence: 1001,"[(The, DT, O), (global, JJ, O), (financial, JJ...",The global financial crisis has left Iceland '...,DT JJ JJ NN VBZ VBN NNP POS NN IN NNS .,O O O O O O B-org O O O O O,"[The, global, financial, crisis, has, left, Ic...","[O, O, O, O, O, O, B-org, O, O, O, O, O]"


In [ ]:
agg_data['len_sentence']=agg_data['tokenised_sentences'].apply(lambda x:len(x))
agg_data['len_tag']=agg_data['tag_list'].apply(lambda x:len(x))
agg_data['is_equal']=agg_data.apply(lambda row:1 if row['len_sentence']==row['len_tag'] else 0,axis=1)
agg_data['is_equal'].value_counts()

1    9967
0       1
Name: is_equal, dtype: int64

In [ ]:
agg_data=agg_data[agg_data['is_equal']!=0]

In [ ]:
agg_data.shape

(9967, 10)

In [ ]:
sentences_list=agg_data['Sentence'].tolist()
tags_list=agg_data['tag_list'].tolist()

print("Number of Sentences in the Data ",len(sentences_list))
print("Are number of Sentences and Tag list equal ",len(sentences_list)==len(tags_list))

Number of Sentences in the Data  9967
Are number of Sentences and Tag list equal  True


In [ ]:
sentences_list=agg_data['Sentence'].tolist()
tags_list=agg_data['tag_list'].tolist()

print("Number of Sentences in the Data ",len(sentences_list))
print("Are number of Sentences and Tag list equal ",len(sentences_list)==len(tags_list))

Number of Sentences in the Data  9967
Are number of Sentences and Tag list equal  True


In [ ]:
tags_list[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
tokeniser= tf.keras.preprocessing.text.Tokenizer(lower=False,filters='')

tokeniser.fit_on_texts(sentences_list)

In [ ]:
print("Vocab size of Tokeniser ",len(tokeniser.word_index)+1) ## Adding one since 0 is reserved for padding

Vocab size of Tokeniser  16247


In [ ]:
tokeniser.index_word[326]

'Earlier'

In [ ]:
encoded_sentence=tokeniser.texts_to_sequences(sentences_list)
print("First Original Sentence ",sentences_list[0])
print("First Encoded Sentence ",encoded_sentence[0])
print("Is Length of Original Sentence Same as Encoded Sentence ",len(sentences_list[0].split())==len(encoded_sentence[0]))
print("Length of First Sentence ",len(encoded_sentence[0]))

First Original Sentence  Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
First Encoded Sentence  [1096, 5, 1220, 16, 2023, 250, 622, 6, 520, 1, 174, 4, 56, 8, 860, 1, 941, 5, 189, 110, 21, 15, 52, 2]
Is Length of Original Sentence Same as Encoded Sentence  True
Length of First Sentence  24


In [ ]:
tags=list(set(data['Tag'].values))
print(tags)
num_tags=len(tags)
print("Number of Tags ",num_tags)

tags_map={tag:i for i,tag in enumerate(tags)}
print("Tags Map ",tags_map)

['B-per', 'I-gpe', 'I-tim', 'I-eve', 'B-nat', 'I-org', 'O', 'I-geo', 'B-org', 'B-geo', 'B-tim', 'B-art', 'I-nat', 'B-eve', 'B-gpe', 'I-per', 'I-art']
Number of Tags  17
Tags Map  {'B-per': 0, 'I-gpe': 1, 'I-tim': 2, 'I-eve': 3, 'B-nat': 4, 'I-org': 5, 'O': 6, 'I-geo': 7, 'B-org': 8, 'B-geo': 9, 'B-tim': 10, 'B-art': 11, 'I-nat': 12, 'B-eve': 13, 'B-gpe': 14, 'I-per': 15, 'I-art': 16}


In [ ]:
reverse_tag_map={v: k for k, v in tags_map.items()}

In [ ]:
encoded_tags=[[tags_map[w] for w in tag] for tag in tags_list]
print("First Sentence ",sentences_list[0])
print('First Sentence Original Tags ',tags_list[0])
print("First Sentence Encoded Tags ",encoded_tags[0])
print("Is length of Original Tags and Encoded Tags same ",len(tags_list[0])==len(encoded_tags[0]))
print("Length of Tags for First Sentence ",len(encoded_tags[0]))

First Sentence  Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .
First Sentence Original Tags  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', 'O', 'O', 'O', 'O', 'B-gpe', 'O', 'O', 'O', 'O', 'O']
First Sentence Encoded Tags  [6, 6, 6, 6, 6, 6, 9, 6, 6, 6, 6, 6, 9, 6, 6, 6, 6, 6, 14, 6, 6, 6, 6, 6]
Is length of Original Tags and Encoded Tags same  True
Length of Tags for First Sentence  24


In [ ]:
max_sentence_length=max([len(s.split()) for s in sentences_list])
print(max_sentence_length)

70


In [ ]:
max_len=128
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

padded_encoded_sentences=pad_sequences(maxlen=max_len,sequences=encoded_sentence,padding="post",value=0)
padded_encoded_tags=pad_sequences(maxlen=max_len,sequences=encoded_tags,padding="post",value=tags_map['O'])

print("Shape of Encoded Sentence ",padded_encoded_sentences.shape)
print("Shape of Encoded Labels ",padded_encoded_tags.shape)

print("First Encoded Sentence Without Padding ",encoded_sentence[0])
print("First Encoded Sentence with padding ",padded_encoded_sentences[0])
print("First Sentence Encoded Label without Padding ",encoded_tags[0])
print("First Sentence Encoded Label with Padding ",padded_encoded_tags[0])

Shape of Encoded Sentence  (9967, 128)
Shape of Encoded Labels  (9967, 128)
First Encoded Sentence Without Padding  [1096, 5, 1220, 16, 2023, 250, 622, 6, 520, 1, 174, 4, 56, 8, 860, 1, 941, 5, 189, 110, 21, 15, 52, 2]
First Encoded Sentence with padding  [1096    5 1220   16 2023  250  622    6  520    1  174    4   56    8
  860    1  941    5  189  110   21   15   52    2    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
First Sentence Encoded Label without Padding  [6, 6, 6, 6, 6, 6, 9, 6, 6, 6, 6, 6, 9, 6, 6, 6

In [ ]:
target= [to_categorical(i,num_classes = num_tags) for i in  padded_encoded_tags]
print("Shape of Labels  after converting to Categorical for first sentence ",target[0].shape)

Shape of Labels  after converting to Categorical for first sentence  (128, 17)


In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val_test,y_train,y_val_test = train_test_split(padded_encoded_sentences,target,test_size = 0.3,random_state=42)
X_val,X_test,y_val,y_test = train_test_split(X_val_test,y_val_test,test_size = 0.2,random_state=42)
print("Input Train Data Shape ",X_train.shape)
print("Train Labels Length ",len(y_train))
print("Input Test Data Shape ",X_test.shape)
print("Test Labels Length ",len(y_test))

print("Input Validation Data Shape ",X_val.shape)
print("Validation Labels Length ",len(y_val))

Input Train Data Shape  (33568, 128)
Train Labels Length  33568
Input Test Data Shape  (2878, 128)
Test Labels Length  2878
Input Validation Data Shape  (11509, 128)
Validation Labels Length  11509


In [ ]:
print("Shape of First Sentence -Train",X_train[0].shape)
print("Shape of First Sentence Label  -Train",y_train[0].shape)

Shape of First Sentence -Train (128,)
Shape of First Sentence Label  -Train (128, 17)


**Building the Model using LSTM**

In [ ]:
from tensorflow.keras import Model,Input
from tensorflow.keras.layers import LSTM,Embedding,Dense
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D,Bidirectional

In [ ]:
embedding_dim=128
vocab_size=len(tokeniser.word_index)+1
lstm_units=128
max_len=128

input_word = Input(shape = (max_len,))
model = Embedding(input_dim = vocab_size+1,output_dim = embedding_dim,input_length = max_len)(input_word)

model = LSTM(units=embedding_dim,return_sequences=True)(model)
out = TimeDistributed(Dense(num_tags,activation = 'softmax'))(model)
model = Model(input_word,out)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 128)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 128, 128)          4501888   
_________________________________________________________________
lstm (LSTM)                  (None, 128, 128)          131584    
_________________________________________________________________
time_distributed (TimeDistri (None, 128, 17)           2193      
Total params: 4,635,665
Trainable params: 4,635,665
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train,np.array(y_train),validation_data=(X_val,np.array(y_val)),batch_size = 32,epochs = 3)

Epoch 1/3
1049/1049 [==============================] - 220s 208ms/step - loss: 0.1034 - accuracy: 0.9799 - val_loss: 0.0413 - val_accuracy: 0.9900
Epoch 2/3
1049/1049 [==============================] - 218s 207ms/step - loss: 0.0277 - accuracy: 0.9926 - val_loss: 0.0252 - val_accuracy: 0.9928
Epoch 3/3
1049/1049 [==============================] - 217s 207ms/step - loss: 0.0185 - accuracy: 0.9944 - val_loss: 0.0232 - val_accuracy: 0.9931


**Evaluating the Model**

In [ ]:
preds=model.predict(X_test)

In [ ]:
def evaluatePredictions(test_data,preds,actual_preds):
    print("Shape of Test Data Array",test_data.shape)
    y_actual=np.argmax(np.array(actual_preds),axis=2)
    y_pred=np.argmax(preds,axis=2)
    num_test_data=test_data.shape[0]
    print("Number of Test Data Points ",num_test_data)
    data=pd.DataFrame()
    df_list=[]
    for i in range(num_test_data):
        test_str=list(test_data[i])
        df=pd.DataFrame()
        df['test_tokens']=test_str
        df['tokens']=df['test_tokens'].apply(lambda x:tokeniser.index_word[x] if x!=0 else '<PAD>')
        df['actual_target_index']=list(y_actual[i])
        df['pred_target_index']=list(y_pred[i])
        df['actual_target_tag']=df['actual_target_index'].apply(lambda x:reverse_tag_map[x])
        df['pred_target_tag']=df['pred_target_index'].apply(lambda x:reverse_tag_map[x])
        df['id']=i+1
        df_list.append(df)
    data=pd.concat(df_list)
    pred_data=data[data['tokens']!='<PAD>']
    accuracy=pred_data[pred_data['actual_target_tag']==pred_data['pred_target_tag']].shape[0]/pred_data.shape[0]


    return pred_data,accuracy

In [ ]:
pred_data,accuracy=evaluatePredictions(X_test,preds,y_test)

Shape of Test Data Array (2878, 128)
Number of Test Data Points  2878


**calculate the precision and recall of each Class - and also the F-Score**

In [ ]:
y_pred=pred_data['pred_target_tag'].tolist()
y_actual=pred_data['actual_target_tag'].tolist()

In [ ]:
print(classification_report(y_actual,y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00        25
       B-eve       1.00      0.21      0.35        19
       B-geo       0.82      0.87      0.84      2190
       B-gpe       0.93      0.91      0.92       909
       B-nat       0.00      0.00      0.00         5
       B-org       0.74      0.57      0.64      1170
       B-per       0.82      0.82      0.82      1026
       B-tim       0.91      0.81      0.86      1244
       I-art       0.00      0.00      0.00        13
       I-eve       0.00      0.00      0.00        17
       I-geo       0.81      0.75      0.78       419
       I-gpe       1.00      0.28      0.43        18
       I-nat       0.00      0.00      0.00         1
       I-org       0.82      0.71      0.76       995
       I-per       0.86      0.89      0.88      1090
       I-tim       0.73      0.65      0.69       386
           O       0.98      0.99      0.99     53126

    accuracy              

In [ ]:
pred_data[pred_data['actual_target_tag']=="B-art"]

,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
9,9370,Vioxx,13,10,B-art,B-org,27
11,3852,Camp,13,2,B-art,B-per,223
7,25979,alHurra,13,6,B-art,I-org,596
29,8157,Please,13,4,B-art,O,690
31,8157,Please,13,4,B-art,O,690
33,8157,Please,13,4,B-art,O,690
4,7382,Telegraph,13,8,B-art,B-geo,734
30,4008,Nuclear,13,10,B-art,B-org,776
5,30307,mon,13,4,B-art,O,858
5,20253,Soprano,13,2,B-art,B-per,1550


In [ ]:
pred_data[pred_data['actual_target_tag']=="B-nat"]

,test_tokens,tokens,actual_target_index,pred_target_index,actual_target_tag,pred_target_tag,id
11,950,Katrina,3,4,B-nat,O,637
6,4013,Rita,3,8,B-nat,B-geo,749
18,10325,H5N2,3,4,B-nat,O,1508
18,10325,H5N2,3,4,B-nat,O,1717
24,827,Hurricane,3,4,B-nat,O,2205
